In [1]:
import sys
# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

In [3]:
from data_providers import ClickHouseConnector
from data_providers import DbDataProviderRT5

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from train_tools import plot_and_go
from basic_application import with_debug_time

%matplotlib notebook
db_connect_params = {
    "host" : "217.197.116.177",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt5"
}

In [4]:
import time
import numpy as np
import pandas as pd

## DatasetDownload

In [8]:
path = "/Users/alex/Dev_projects/RT/model_train/datasets/rt5/03.BTC_USDT-300-train.csv"
dataset = pd.read_csv(path, index_col='ts')
dataset.head()

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1662185340,2022-09-03 06:09:00,19929.31,19928.361,"{'19929.31': 0.5, '19931.24': 0.25, '19932.14'...","{'19928.36': 0.25, '19926.23': 0.5, '19924.98'...",0.177276,0.164399,0.341675,9,10,19
1662185640,2022-09-03 06:14:00,19915.01,19913.000,"{'19915.01': 0.492324, '19920.6': 0.25, '19922...","{'19913': 0.25, '19911.03': 0.226976, '19910.9...",0.018660,0.091484,0.110144,6,10,16
1662185940,2022-09-03 06:19:00,19910.93,19908.920,"{'19910.93': 0.467253, '19913.63': 0.25, '1991...","{'19908.92': 0.25, '19906.6': 0.25, '19903.85'...",0.108600,0.187092,0.295692,9,7,16
1662186240,2022-09-03 06:24:00,19836.08,19826.230,"{'19836.08': 0.5, '19838.07': 0.25, '19845.43'...","{'19826.23': 0.25, '19825.24': 0.14, '19825.23...",0.548822,0.795283,1.344105,16,11,27
1662186540,2022-09-03 06:29:00,19859.43,19858.441,"{'19859.43': 0.5, '19865.16': 0.25, '19867.4':...","{'19858.44': 0.25, '19857.44': 0.5, '19855.44'...",0.374214,0.659355,1.033569,8,13,21


# DataPointV3

In [181]:

"""
Класс DataPoint реализован, чтобы передать текущее представление точки данных в core_v2.

Для обычных фичей достаточно методов get_value и get_values со стандартной длиной выборки, равной observation_len
В некоторых случаях, возможно, потребуется получать весь хвост данных, чтобы взять объективное среднее для нормализации.

Дополнительные методы типа get_current_index, и get_value со специфическим индексом нужны в фиче с историей.
И то - для текущей точки достаточно current_index, а в init'е - наверное, можно использовать относительное указание индекса

Т.к. пока нет необходимости работать с историей (сделал быстро вычисление на лету) - то достаточно двух базовых методов

Второй поинт - пока нет работы с future_points - отрицательную индексацию модно исключить. 
Это должно повысить производлительность, но, возможно, игра не стоит свеч и можно все оставить в одном классе, надо тестить.


Из этой реализации убраны некоторые методы, которые были в старой в силу их ненадобности.'
"""
import numpy as np

from basic_application import with_exception


class DataPointError(Exception):
    pass


class DataPoint:
    """DataPointV2, описание смотри в модуле."""
    def __init__(self, data, observation_len=5, future_points=0):
        self.data = data
        # Длина наблюдения. Для разных step_factor включает в себя разное количество точек данных.
        self.observation_len = observation_len
        # Количество точек данных в будущем 
        self.future_points = future_points
        # Количество точек данных, доступных для построения observation с учетом всех доступных step_factors
        self.tail_points = len(self.data) - self.future_points
        
        # В "абстрактных" тестовых сценариях присутствуют кейсы, где дата поинт состоит из одной точки. 
        # На синтетических и реальных данных такого нет, observation_len всегда больше 1
        if self.data.shape[0] > 1:
            self.period = data.index[1] - data.index[0]
        else:
            self.period = 1

        # Верхушка observation = текущая точка данных. 
        #Так это последняя точка данных, но future_points может ее сместить ближе к началу.
        self.cursor = data.shape[0] - self.future_points - 1  # Курсор про номер позии
        self.index = data.index.values[self.cursor]           # Индекс про индекс
        
    def _get_num(self, step_factor, num):
        "Вычисляет количество точек наблюдения и возволяет в большей части случаев исключить эту логику из потребителя"
        if num is None:
            # Базовый сценарий
            return self.observation_len 
        elif num == -1:
            # Вычисление позволяет передать максимальное количество точек для вычисления value со step_factor выше 1.
            return self.tail_points // (self.observation_len*step_factor) * (self.observation_len * step_factor)
        else:
            return num
        
    def get_index_slice(self, step_factor=1, num=None):
        """Возвращает все курсоры, в том числе расположенные между реперными точками"""
        num = self._get_num(step_factor, num)

        if num >= 0:
            # верхний индекс на единицу выше, т.к. это элемент не захватывается при слайсинге
            up_bound = self.cursor + 1
            low_bound = up_bound - num * step_factor
        else:
            # Отрицательные значения нужны при работе с future points. Надо протестить производительномть, можент быть выпилить эту часть.
            low_bound = self.cursor + 1
            up_bound = low_bound - num * step_factor
        return low_bound, up_bound
            
    def get_values(self, name, step_factor=1, num=None, agg="average"):
        """Возвращает значений точек данных заданной длины и агрегированное для step_factor выше 1
        """
        num = self._get_num(step_factor, num)
        values = self.data.loc[:, name].values
        low_bound, up_bound = self.get_index_slice(step_factor=step_factor, num=num)
        values = values[low_bound : up_bound]
        if step_factor > 1:
            # небольшая оптимизация производительности
            values = getattr(np, agg)(values.reshape(-1, step_factor), axis=1)
        return values
    
    def get_value(self, name, step_factor=1, agg="average"):
        """Метод возвращает одно значение агрегированное для step_factor выше 1"""
        value = self.get_values(name, step_factor=step_factor, agg=agg, num=1)
        return value[0]
    
    def get_value_by_idx(self, name, idx, step_factor=1, agg="average"):
        """
        Метод возвращает одно значение агрегированное для step_factor выше 1 по индексу
        Метод нужен только в сценарии рассчета фичи с историей хранения (для инициализации)
        """
        cursor = np.where(self.data.index.values == idx)[0][0]
        values = self.data.loc[:, name].values
        values = values[cursor - step_factor + 1: cursor + 1]
        if step_factor > 1:
            values = getattr(np, agg)(values.reshape(-1, step_factor), axis=1)
        return values[0]

    def get_current_index(self):
        return self.index
    
    def get_current_cursor(self):
        return self.cursor

In [207]:
dp = DataPoint(dataset.head(10), observation_len=10, future_points=0)

In [208]:
dp.data.index

Int64Index([1662185340, 1662185640, 1662185940, 1662186240, 1662186540,
            1662186840, 1662187140, 1662187440, 1662187740, 1662188040],
           dtype='int64', name='ts')

In [209]:
dp.get_current_index()

1662188040

In [210]:
dp.get_current_cursor()

9

In [211]:
dp.get_index_slice()

(0, 10)

In [215]:
step_factor = 1
s = dp.get_values("highest_bid", step_factor=step_factor)
print(s)

print(dp.data["highest_bid"].values[-dp.observation_len*step_factor:])

[19928.361 19913.    19908.92  19826.23  19858.441 19888.23  19876.55  19869.979 19898.2
 19884.28 ]
[19928.361 19913.    19908.92  19826.23  19858.441 19888.23  19876.55  19869.979 19898.2
 19884.28 ]


In [213]:
step_factor = 1
s = dp.get_value("highest_bid", step_factor=step_factor)
print(s)


19884.28


In [152]:
dp.get_values("highest_bid", step_factor=2, num=3)

array([19300.544 , 19308.3755, 19321.59  ])

In [112]:
dp.data.tail(8)

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1662679440,2022-09-08 23:24:00,19329.950,19329.941,"{'19329.95': 0.46, '19332.01': 0.23, '19332.98...","{'19329.94': 0.46, '19327.99': 0.23, '19326.05...",0.093351,0.332515,0.425866,6,6,12
1662679740,2022-09-08 23:29:00,19328.000,19327.990,"{'19328': 0.493774, '19329.95': 0.197209, '193...","{'19327.99': 0.23, '19326.05': 0.23, '19323.16...",0.147513,0.169983,0.317496,12,10,22
1662680040,2022-09-08 23:34:00,19303.340,19303.328,"{'19303.34': 0.46, '19304.37': 0.23, '19305.33...","{'19303.33': 0.46, '19300.7': 0.229659, '19299...",0.181660,0.208626,0.390286,8,10,18
1662680340,2022-09-08 23:39:00,19300.709,19297.760,"{'19300.71': 0.46, '19302.74': 0.1, '19303.34'...","{'19297.76': 0.321161, '19295.83': 0.116558, '...",0.740906,0.555953,1.296859,14,12,26
1662680640,2022-09-08 23:44:00,19307.469,19307.460,"{'19307.47': 0.23, '19309.41': 0.46, '19311.35...","{'19307.46': 0.46, '19303.33': 0.349732, '1930...",0.659533,0.755492,1.415025,11,13,24
1662680940,2022-09-08 23:49:00,19309.299,19309.291,"{'19309.3': 0.46, '19311.31': 0.23, '19313.25'...","{'19309.29': 0.46, '19308.77': 0.163093, '1930...",0.319304,0.867603,1.186907,4,8,12
1662681240,2022-09-08 23:54:00,19319.270,19317.170,"{'19319.27': 0.451095, '19321.32': 0.23, '1932...","{'19317.17': 0.459211, '19315.01': 0.23, '1931...",1.348322,0.676910,2.025232,17,6,23
1662681540,2022-09-08 23:59:00,19326.020,19326.010,"{'19326.02': 0.293474, '19327.95': 0.229311, '...","{'19326.01': 0.002403, '19325.05': 0.23, '1932...",0.737640,0.495444,1.233084,18,8,26


In [43]:
step_factor = 2

num = 8
num_ = num * step_factor


low_bound, up_bound = dp.get_indexes(step_factor=step_factor, num=num_)
print(low_bound, up_bound)


1618 1650


In [52]:
name = "highest_bid"
values = dp.data.loc[:, name].values
values

array([19928.361, 19913.   , 19908.92 , ..., 19309.291, 19317.17 ,
       19326.01 ])

In [53]:
result = values[low_bound: up_bound]
result

array([19371.479, 19368.92 , 19332.64 , 19329.682, 19334.58 , 19320.7  ,
       19316.15 , 19333.43 , 19342.23 , 19293.602, 19304.32 , 19328.91 ,
       19325.97 , 19300.76 , 19300.012, 19320.   , 19304.08 , 19274.27 ,
       19276.791, 19258.21 , 19278.371, 19281.27 , 19295.58 , 19287.41 ,
       19297.51 , 19316.74 , 19309.201, 19311.629, 19307.49 , 19329.941,
       19327.99 , 19303.328])

In [54]:
agg = "average"
result = getattr(np, agg)(result.reshape(-1, step_factor), axis=1)
result

array([19370.1995, 19331.161 , 19327.64  , 19324.79  , 19317.916 ,
       19316.615 , 19313.365 , 19310.006 , 19289.175 , 19267.5005,
       19279.8205, 19291.495 , 19307.125 , 19310.415 , 19318.7155,
       19315.659 ])

In [209]:
dp.get_indexes()

array([1662807000, 1662807300, 1662807600, 1662807900, 1662808200,
       1662808500, 1662808800, 1662809100, 1662809400, 1662809700])

In [210]:
ss = dp.get_values(name="highest_bid", num=2, as_ndarray=True)
ss

array([21327.541, 21294.281], dtype=float32)

In [211]:
dd = dp.get_value("highest_bid", cursor = 1662797100)
dd

21284.56

In [212]:
dp.get_current_index()

1662809700

In [157]:
ss.diff().dropna().values

array([-20.488281, -33.259766], dtype=float32)

In [148]:
cursor = dp.get_indexes(num=1)
dp.get_values(cursor=cursor, num=dp.hist_len, as_ndarray=False)

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1662799500,2022-09-10 08:45:00,21344.490234,21317.599609,"{'21344.49': 0.096623, '21344.78': 0.212242, '...","{'21317.6': 0.7, '21306.54': 0.234, '21297.13'...",1.143023,0.328697,1.471720,24,16,40
1662799800,2022-09-10 08:50:00,21310.361328,21309.750000,"{'21310.36': 0.204283, '21318': 0.235, '21319....","{'21309.75': 0.670705, '21307.18': 0.013668, '...",0.249728,1.624330,1.874058,9,31,40
1662800100,2022-09-10 08:55:00,21321.199219,21319.800781,"{'21321.2': 0.195339, '21326': 0.232063, '2134...","{'21319.8': 0.211053, '21300.99': 0.386166, '2...",0.834215,0.747824,1.582039,19,31,50
1662800400,2022-09-10 09:00:00,21305.599609,21296.740234,"{'21305.6': 0.196293, '21308.23': 0.210702, '2...","{'21296.74': 0.665715, '21276.67': 0.011152, '...",0.203970,1.093149,1.297119,24,27,51
1662800700,2022-09-10 09:05:00,21292.119141,21291.371094,"{'21292.12': 0.03, '21293.5': 0.235, '21295.63...","{'21291.37': 0.434304, '21283.46': 0.012226, '...",0.047145,0.374906,0.422051,8,38,46
1662801000,2022-09-10 09:10:00,21296.298828,21292.869141,"{'21296.3': 0.123634, '21307.61': 0.004343, '2...","{'21292.87': 0.234758, '21290.43': 0.235, '212...",1.571577,1.358688,2.930265,27,66,93
1662801300,2022-09-10 09:15:00,21313.320312,21312.349609,"{'21313.32': 0.235, '21315.46': 0.235, '21316....","{'21312.35': 0.7, '21311.26': 0.235, '21309': ...",0.777562,1.200628,1.978190,10,59,69
1662801600,2022-09-10 09:20:00,21337.880859,21332.740234,"{'21337.88': 0.03043, '21343.85': 0.235, '2134...","{'21332.74': 0.235, '21328.78': 0.500289, '213...",1.094926,0.835427,1.930353,15,13,28
1662801900,2022-09-10 09:25:00,21353.378906,21327.929688,"{'21353.38': 0.226739, '21355.52': 0.235, '213...","{'21327.93': 0.136858, '21327.12': 0.468816, '...",2.060175,0.863481,2.923656,22,14,36


In [141]:
import json

In [146]:
cursor_prev = dp.get_indexes(num=1)
data_raw = dp.get_values(name="asks", num=1, cursor=cursor_prev, as_ndarray=False)
data_raw

{'21307.77': 0.01,
 '21316.5': 0.189,
 '21321.21': 0.002199,
 '21325.77': 0.7,
 '21327.7': 0.002551,
 '21330.13': 0.00247,
 '21331.14': 0.001882,
 '21333.7': 0.8,
 '21340.02': 0.14,
 '21346': 0.394,
 '21363.4': 0.00354,
 '21365': 0.000411,
 '21400': 0.04,
 '21406': 0.03,
 '21409.16': 0.002787,
 '21414.06': 0.003403,
 '21418.9': 0.001191,
 '21418.94': 0.00116,
 '21421.21': 0.004,
 '21450': 0.008464,
 '21450.93': 3.956,
 '21464.38': 0.003394,
 '21500': 0.001,
 '21514.81': 0.003387,
 '21565.37': 0.003378,
 '21574.82': 0.031452,
 '21576.76': 0.2,
 '21578.64': 0.030488,
 '21587.36': 0.028802,
 '21593.51': 0.061227,
 '21593.55': 0.09926,
 '21594.98': 0.09926,
 '21600': 0.188825,
 '21602.57': 1.964,
 '21604.16': 6.7e-05,
 '21605.92': 0.000852,
 '21616.04': 0.003373,
 '21618.9': 0.012761,
 '21624.38': 0.031118,
 '21624.39': 0.005516,
 '21626.11': 0.031118,
 '21640': 5e-05,
 '21650': 0.089387,
 '21654': 0.52487,
 '21666.84': 0.003361,
 '21672.01': 0.001153,
 '21673.71': 0.009192,
 '21685.05': 0

In [97]:
orderbook_prev = json.loads(data_raw.replace("\'", "\""))

1

In [102]:
np.arange(20,10,-1)

array([20, 19, 18, 17, 16, 15, 14, 13, 12, 11])

In [81]:
current_idx = 1662811200
n=-1


if n > 0:
    period = data.index[0] - data.index[1]
    stop = current_idx + (period * n)
else:
    period = data.index[1] - data.index[0]
    stop = current_idx - (period * n)

idxs = np.arange(current_idx, stop, period)
idxs

array([1662811200])

In [82]:
def get_values(name=None, cursor=None, step_factor=1, num=None):

    if cursor is None:
        cursor = current_idx

    if name is None:
        name = data.columns
    
    if num is None:
        num = 4 # observation 
        
    period = (data.index[1] - data.index[0]) * step_factor

    if num > 0:
        stop = cursor - (period * (num-1))
        idxs = np.arange(stop, cursor+period, period)
    else:
        stop = cursor - (period * num)
        idxs = np.arange(cursor, stop, period)
        
    df = data.loc[idxs, name]
    display(df)
    

In [100]:
def get_indexes(cursor=None, step_factor=1, num=None):
         
    if cursor is None:
        cursor = current_idx

    if num is None:
        num = 4

    period = (data.index[1] - data.index[0]) * step_factor

    if num > 0:
        stop = cursor - (period * (num-1))
        idxs = np.arange(stop, cursor+period, period)
    else:
        stop = cursor - (period * num)
        idxs = np.arange(cursor, stop, period)

    return idxs

get_indexes()

array([1662810300, 1662810600, 1662810900, 1662811200])

## Datapoint Factory

In [9]:
from data_point import DataPointFactory

In [15]:
n_observation_points = 5
n_future_points = 0
n_history_points = 2
period = 300

dpf = DataPointFactory(
    dataset=data, 
    period=period, 
    n_observation_points=n_observation_points,
    n_history_points = n_history_points,
    n_future_points=n_future_points
)


In [16]:
dp = dpf.get_current_step()

In [17]:
dp.__dict__

{'data':                             dt    lowest_ask   highest_bid  \
 ts                                                           
 1662796500 2022-09-10 07:55:00  21352.500000  21352.310547   
 1662796800 2022-09-10 08:00:00  21353.130859  21274.349609   
 1662797100 2022-09-10 08:05:00  21326.000000  21284.560547   
 1662797400 2022-09-10 08:10:00  21325.759766  21293.109375   
 1662797700 2022-09-10 08:15:00  21304.380859  21281.400391   
 1662798000 2022-09-10 08:20:00  21286.029297  21271.300781   
 1662798300 2022-09-10 08:25:00  21314.351562  21296.478516   
 
                                                          asks  \
 ts                                                              
 1662796500  {'21352.5': 0.394, '21379.27': 0.036275, '2138...   
 1662796800  {'21353.13': 0.480706, '21354.98': 0.280799, '...   
 1662797100  {'21326': 0.690315, '21339.2': 0.020079, '2134...   
 1662797400  {'21325.76': 0.689794, '21345.84': 0.235, '213...   
 1662797700  {'21304.38': 0

In [61]:
dp, done = dpf.get_next_step()
print(done)

True


In [62]:
dp.data

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1662809400,2022-09-10 11:30:00,21335.001953,21327.541016,"{'21335': 0.1, '21335.01': 0.00098, '21335.79'...","{'21327.54': 0.14, '21327.53': 0.55, '21322.34...",1.316541,1.773979,3.090520,21,30,51
1662809700,2022-09-10 11:35:00,21307.771484,21294.281250,"{'21307.77': 0.01, '21316.5': 0.189, '21321.21...","{'21294.28': 0.14, '21294.27': 0.469515, '2129...",1.292653,1.073682,2.366335,18,17,35
1662810000,2022-09-10 11:40:00,21298.380859,21290.628906,"{'21298.38': 0.01, '21298.73': 0.00247, '21299...","{'21290.63': 0.69, '21290.6': 1.413195, '21280...",1.122753,1.586319,2.709072,15,17,32
1662810300,2022-09-10 11:45:00,21300.250000,21298.650391,"{'21300.25': 0.00073, '21300.26': 0.15, '21307...","{'21298.65': 0.14, '21297.14': 0.69, '21292.09...",1.888513,1.531294,3.419807,19,19,38
1662810600,2022-09-10 11:50:00,21295.941406,21287.980469,"{'21295.94': 0.01, '21299.02': 0.15, '21308.22...","{'21287.98': 0.14, '21287.97': 0.55, '21287.96...",2.045997,1.473329,3.519326,26,15,41
1662810900,2022-09-10 11:55:00,21279.521484,21277.070312,"{'21279.52': 0.001214, '21282.04': 0.15, '2128...","{'21277.07': 0.55, '21273.72': 1.426844, '2126...",0.661370,1.257566,1.918936,10,12,22
1662811200,2022-09-10 12:00:00,21288.179688,21279.009766,"{'21288.18': 1.370121, '21288.19': 0.7, '21289...","{'21279.01': 0.000223, '21273.43': 0.14, '2127...",1.432162,1.484592,2.916754,19,21,40
